# Example usage

To use `linreg_ally` in a project:

In [3]:
import linreg_ally

print(linreg_ally.__version__)

0.1.0


In [ ]:
# Imports

In [ ]:
# Paramveer - EDA

In [ ]:
# Alex - VIF 

In [ ]:
# Cheng - model fitting

In [48]:
# DELETE THIS CODE CHUNK IF CHENG USES THE SAME ARGUMENTS FOR HIS RUN_LINEAR_REGRESSION FUNCTION
# Merari - plot (using Cheng's function outputs)
from vega_datasets import data
from linreg_ally.models import run_linear_regression

df = data.cars()
df = df[['Horsepower', 'Displacement']].dropna()

# Define parameters for run_linear_regression
dataframe = df
target_column = "Horsepower"
numeric_feats = ["Displacement"] 
categorical_feats = []  # No categorical features in this case
drop_feats = None  # No columns to drop
random_state = 123

model_results = run_linear_regression(
    dataframe=dataframe,
    target_column=target_column,
    numeric_feats=numeric_feats,
    categorical_feats=categorical_feats,
    drop_feats=drop_feats,
    random_state=random_state
)


Model Summary
------------------------
Test r2: 0.785
Test neg_mean_squared_error: 345.987


## Checking Normality and Homoscedasticity of Residuals

A linear regression model assumes that residuals are normally distributed and have constant variance (homoscedasticity). To check whether these assumptions are met, we use the `qq_and_residuals_plot` function. This function generates:

1. A Quantile-Quantile (Q-Q) plot to assess the normality of residuals.
2. A Residuals vs. Fitted Values plot to check for homoscedasticity.

The `qq_and_residuals_plot` function takes two parameters: `y_actual` and `y_predicted`. These values were extracted from the linear regression model we previously created.

In [49]:
# Unpack the returned values correctly
best_model, X_train, X_test, y_train, y_test, scores = model_results

# y_actual is y_test (true labels)
y_actual = y_test

# y_predicted is obtained by predicting on X_test
y_predicted = best_model.predict(X_test)


Now that `y_actual` and `y_predicted` have been extracted, let's pass these parameters to the `qq_and_residuals_plot` function.

In [50]:
from linreg_ally.plotting import qq_and_residuals_plot

qq_and_residuals_plot(y_actual, y_predicted)

alt.HConcatChart(...)

## Interpreting the Q-Q Plot

If the Q-Q plot shows a significant deviation from the red dashed line (which represents perfect normality), the residuals are not normally distributed. In our plot, a few points deviate from the line at the tails, suggesting potential skewness or outliers. However, since these deviations are minor, we can conclude that the residuals are approximately normal.

## Interpreting the Residuals vs. Fitted Values Plot

For the homoscedasticity assumption to hold, residuals should be randomly scattered around the red dashed line in the Residuals vs. Fitted Values plot. This would indicate that residual variance remains constant across all fitted values (homoscedasticity).

However, in our case, residuals cluster at different fitted value ranges, suggesting that the variance is not constant (heteroscedasticity).

## Implications of Assumption Violations

If the normality assumption is violated:
Ordinary Least Squares (OLS) regression still produces best linear unbiased estimates (BLUE) as long as independence and homoscedasticity hold. However, hypothesis tests and confidence intervals may be misleading if residuals deviate significantly from normality.

If the homoscedasticity assumption is violated:
You can still fit a linear regression model, but you should interpret results with caution. The estimated coefficients remain unbiased, but standard errors and p-values become unreliable, affecting statistical inference.

## Conclusion

The `qq_and_residuals_plot` function is a valuable tool for assessing the normality and homoscedasticity assumptions in linear regression. If these assumptions are violated, you should consider corrective measures such as:

- Transforming variables (e.g., logarithmic transformation),
- Using robust standard errors, or
- Exploring alternative models (e.g., weighted least squares, generalized least squares).